# Euler buckling (Clamped-Clamped end conditions)

<img src="http://www.gunt.de/images/datasheet/1530/WP-121-Demonstration-of-Euler-buckling-gunt-1530-zeichnung.jpg" width="400" align="right">

<p>A vertical beam submitted to a vertical force can exhibit an instability.   
The critical load for instability depends on the end conditions of the beam.   
The picture displays fours different end conditions.   

The Euler buckling problem is formulated as a constrained optimization.   
The total energy, potential + deformation, is minimum for stable equilibrium.   
The constrains are the end conditions and the geometric equations of the beam.   
The   <a href="https://www.amazon.com/Optimization-Modeling-Python-Springer-Applications/dp/3319588192/ref=sr_1_1?ie=UTF8&amp;qid=1503778130&amp;sr=8-1&amp;keywords=pyomo">Pyomo Optimization Modeling in Python</a>   package is used to solve the problem.</p>
**Clamped-Clamped** end conditions correspond to the **3th situation** from left in the picture.

**Dependencies:**   
Pyomo and Ipopt, for optimization, are installed automatically from the notebooks if needed.   
Bokeh and Holoviews, for graphics, need to be installed beforehands.

### install  pyomo,  ipopt  and  glpk

In [ ]:
%%time
import subprocess
try:
    import pyomo
    print("Pyomo already installed")
except ImportError:
    print(subprocess.run("pip install pyomo", shell=True))
    print(subprocess.run("conda install ipopt_bin -c cachemeorg", shell=True))
    print(subprocess.run("conda install glpk -c cachemeorg", shell=True))
    
try:
    import holoviews
    print("holoviews already installed")
except ImportError:
    print("Installing holoviews")
    print(subprocess.run("conda install -c ioam holoviews bokeh", shell=True))  

### import librairies

In [ ]:
from pyomo.environ import *
import holoviews as hv
hv.extension('bokeh')

### a function that defines and solves the Euler buckling problem
    
    alpha:       load of the beam, normalized
    nSegments:   number of segments used in the model

In [ ]:
import ipywidgets as widgets
from IPython.core.display import display

class myBeam:
    def __init__(self, nSegments=400):
        m = ConcreteModel() 
        m.N = nSegments
        m.h = 1/nSegments
        m.Segments = RangeSet(1, m.N)
        m.Nodes    = RangeSet(0, m.N)
        
        m.alpha = Param(mutable=True)
        m.alpha = 0
        m.x = Var(m.Nodes, bounds=(-5,5), initialize=0)
        m.y = Var(m.Nodes, bounds=(-5,5), initialize=0)
        m.t = Var(m.Nodes, bounds=(-5,5), initialize=1)
        m.u = Var(m.Nodes,                initialize=0)
        m.ED = Var()
        m.EP = Var()
        m.ET = Var()

        m.eqs = ConstraintList()
        for i in m.Segments: m.eqs.add( m.x[i] - m.x[i-1] == (sin(m.t[i]) + sin(m.t[i-1])) * m.h/2 )
        for i in m.Segments: m.eqs.add( m.y[i] - m.y[i-1] == (cos(m.t[i]) + cos(m.t[i-1])) * m.h/2 )
        for i in m.Segments: m.eqs.add( m.t[i] - m.t[i-1] == (m.u[i]      + m.u[i-1])      * m.h/2 )

        m.eqs.add( m.ED == sum( 0.5*m.h * (m.u[i]**2 + m.u[i-1]**2)  for i in m.Segments ) )
        m.eqs.add( m.EP == m.alpha * m.y[m.N] )
        m.eqs.add( m.ET == m.ED + m.EP )

        m.eqs.add( m.x[0]     == 0)
        m.eqs.add( m.y[0]     == 0)
        m.eqs.add( m.t[0]     == 0)
        m.eqs.add( m.x[m.N]   == 0.0001)
        m.eqs.add( m.t[m.N]   == 0)
        
        m.obj = Objective(expr=m.ET, sense=minimize)
        
        self.m = m
        
    def load(self, alpha):
        m = self.m
        m.alpha = alpha
        results = SolverFactory('ipopt').solve(m)
        s = 1
        if m.x[1].value < 0: s = -1
        xval=[m.x[i].value*s for i in m.Nodes]
        yval=[m.y[i].value   for i in m.Nodes]
        tval=[m.t[i].value   for i in m.Nodes]
        uval=[m.u[i].value   for i in m.Nodes]
        dx = max(abs(max(xval)),abs(min(xval)))
        dy = yval[m.N]        
        return [xval, yval, tval, uval, dx, dy, m.ED.value, m.EP.value, m.ET.value]

    def loop(self, alphas):
        steps  = range(0,len(alphas))
        bar = widgets.IntProgress(value=0, min=0, max=len(alphas), description='Progress:')
        display(bar)
        a, x, y, t, u, dx, dy, ed, ep, et = [[[] for i in steps] for j in range(0,10)]                      # storage
        for i in steps:
            x[i], y[i], t[i], u[i], dx[i], dy[i], ed[i], ep[i], et[i] = self.load(alphas[i])
            bar.value = i
        return {"load":alphas,"x":x,"y":y,"t":t,"u":u,"dx":dx,"dy":dy,"ed":ed,"ep":ep,"et":et}       

### calculating beam stable deformation for different loads

In [ ]:
%%time
import numpy as np
def frg(x0, x1, dx, ndigits=3): return set(np.arange(x0,x1+dx,dx).round(6).tolist())

loads = list( set().union(frg(0,250,5), frg(75,90,1), frg(78,80,0.05), frg(170,175,1)) )
loads.sort()
test = myBeam().loop(loads)

### deformation and energies as functions of load

In [ ]:
optionC  = {'Curve':{'style': {"line_width":3, 'line_alpha':1}, 'plot': {'height':400, 'width':400, 'show_grid':True, 'tools':['hover']}}}
optionC1 = {'Curve':{'style': {"line_width":8, 'line_alpha':0.5}, 'plot': {'height':400, 'width':400, 'show_grid':True, 'tools':['hover']}}}
optionO  = {'Overlay':{'style': {}, 'plot': {'legend_position':'bottom_left', 'height':400, 'width':400, 'show_grid':True, 'tools':['hover']}}}

p1 = hv.Curve(test, kdims=['load'], vdims=['dx'], label="horizontal displacement")(optionC)
p2 = hv.Curve(test, kdims=['load'], vdims=['dy'], label="vertical displacement")(optionC)
p3 = hv.Curve(test, kdims=['load'], vdims=['ed'], label="deformation energy")(optionC)
p4 = hv.Curve(test, kdims=['load'], vdims=['ep'], label="potential energy")(optionC)
p5 = hv.Curve(test, kdims=['load'], vdims=['et'], label="total energy")(optionC1)

((p1*p2) + (p5*p4*p3))(optionO)
# hv.help(p1)

### shape as function of load

In [ ]:
options = {'Points':{
                'style': {'size':0.5, 'color':'red'}, 
                'plot': {'color_index':2, 'height':400, 'width':400, 'show_grid':True, 'tools':['hover']}}}

def aplot(a):
    i = test["load"].tolist().index(a)
    x = test["x"][i]
    y = test["y"][i]
    u = test["u"][i]
    u = np.abs(np.array(u)) / umax
    c1 = hv.Points((x,y,u), vdims=['z'])
    c2 = hv.Points(([-1,1],[-1,1],[0,1]), vdims=['z'])
    c = c1*c2
    return c(options)

umax = np.abs(np.array(test["u"])).max()

dmap = hv.DynamicMap(aplot, kdims=['load'])
dmap.redim.values(load=test["load"])